In [1]:
import pandas as pd
from utility import cache_data, upload_data_s3

In [3]:
### Load Strava Data

data_path = "/Users/hadid/GitHub/ETL/data/raw_data/strava_data.xlsx"
strava_data = pd.read_excel(data_path)
strava_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1231 entries, 0 to 1230
Data columns (total 94 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   resource_state                                   1231 non-null   int64  
 1   name                                             1231 non-null   object 
 2   distance                                         1231 non-null   float64
 3   moving_time                                      1231 non-null   int64  
 4   elapsed_time                                     1231 non-null   int64  
 5   total_elevation_gain                             1231 non-null   float64
 6   type                                             1231 non-null   object 
 7   sport_type                                       1231 non-null   object 
 8   workout_type                                     120 non-null    float64
 9   id                            

In [9]:
upload_data_s3.post_data_to_s3(strava_data, "strava_activity_data")

[2024-06-12 17:58:22] [INFO]  Starting upload to S3 bucket: etl-5h3gn2wqhzfd, data source: strava_activity_data
[2024-06-12 17:58:26] [INFO]  DataFrame uploaded to S3 bucket 'etl-5h3gn2wqhzfd' as 'strava/strava_activity_data_2024-06-12_17-58-22.csv'


True

In [4]:
### Save IDs to Redis

# Extract activity IDs
print(strava_data["id"].head())
activity_ids = strava_data["id"]
print(activity_ids)
print(type(activity_ids))

0    11715123516
1    11714332345
2    11714332359
3    11706971235
4    11706971242
Name: id, dtype: int64
0       11715123516
1       11714332345
2       11714332359
3       11706971235
4       11706971242
           ...     
1226     7613744359
1227     7613744403
1228     7613744400
1229     7613744405
1230     7613744483
Name: id, Length: 1231, dtype: int64
<class 'pandas.core.series.Series'>


In [5]:
# Cache activity IDs in Redis
cache_data.cache_ids("strava_activity_ids", activity_ids)

[2024-06-24 17:05:52] [INFO]  Updating cached strava_activity_ids in Redis...
[2024-06-24 17:05:52] [INFO]  Successfully updated cached strava_activity_ids, added 1231 new IDs


In [7]:
# Initialise cache and get cached activity IDs from Redis
cached_ids = cache_data.get_cached_ids("strava_activity_ids")
print(len(cached_ids))
# print(f"Cached activity IDs: {cached_ids}")
# print(f"Data Type: {type(cached_ids)}")

[2024-06-24 17:06:26] [INFO]  Fetching cached strava_activity_ids from Redis...
[2024-06-24 17:06:26] [INFO]  Successfully fetched 1231 IDs


1231
